# Setup

In [1]:
%pip install OpenAI

Note: you may need to restart the kernel to use updated packages.


In [2]:

from IPython.display import display, Markdown
import os

### Pretty printing help

In [3]:
import json

def show_json(obj):
    """
    Display the JSON representation of the given object.

    Parameters:
    obj (object): The object to display as JSON.

    Returns:
    None
    """
    display(json.loads(obj.model_dump_json()))

### Load environment variables

In [4]:
from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

## Choose the Example

The following will let the user switch to a different example to run through the pipeline.

In [2]:
import ipywidgets as widgets

def set_destinations(b):
  # This function will be called when the button is clicked
  global choice, job_description_file, company_file, destination_directory, html_destination, pdf_destination, markdown_destination, word_destination
  choice = choice_widget.value
  print("You selected: " + choice)
  # source files
  job_description_file = "examples/" + choice + "/job-description.md"
  company_file = "examples/" + choice + "/about-company.md"

  # destination files
  destination_directory = "examples/" + choice + "/"
  html_destination = destination_directory + 'playbook.html'
  pdf_destination = destination_directory + 'playbook.pdf'
  markdown_destination = destination_directory + 'playbook.md'
  word_destination = destination_directory + 'playbook.docx'


# Which example to run
choice_widget = widgets.Dropdown(
  options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator','growth-manager'],
  value='medical-assistant',
  description='Choose an example:'
)
choice = choice_widget.value

# trigger the button to run the function
set_destinations(choice_widget)

button = widgets.Button(description="Confirm")


button.on_click(set_destinations)

display(choice_widget, button)



You selected: medical-assistant


Dropdown(description='Choose an example:', options=('medical-assistant', 'infra-engineer', 'customer-success-m…

Button(description='Confirm', style=ButtonStyle())

### Job Description

Read the job description into a variable.

In [15]:
job_description_file = "examples/" + choice + "/job-description.md"
with open(job_description_file, "r") as file:
    job_description = file.read()

print(job_description)

## Job Description

**Y Combinator - Infrastructure Software Engineer**

Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benefits and unlimited vacation – while working on a small, fast-moving and high-impact team.

In your first month, you might:

- M

### Company Description

Read the company description into a variable.

In [16]:

company_file = "examples/" + choice + "/about-company.md"

with open(company_file, "r") as file:
    company_description = file.read()

print(company_description)

## Y Combinator Mission, Vision, and Values

### Mission

Y Combinator's mission is to increase the number of startups by assisting founders in their creation. The organization aims to get startups through their initial phase, often helping them build something impressive enough to raise significant amounts of money on a larger scale.

### Vision

Y Combinator envisions a world where startups are the norm rather than the exception. They believe that startups are, on balance, a good thing, as their founders and early employees can be much more productive than they would be working for an established company. Y Combinator's vision is to apply mass production techniques to startup funding, making the process more efficient and beneficial for founders.

### Values

Y Combinator values integrity, respect, and accountability. They believe these traits are critical for founders to possess. The organization is built on a network of trust, with its strength and value hinging on the trustworthin

# Develop top-level list of competencies

## Setup OpenAI

In [17]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

## Get Draft List of Competencies

In [18]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as competencies for a hiring team to assess in a candidate. 
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the key competencies for the job description as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "job_level": "Google Job Level (1-10)",
  "thinking": "Your thoughts about how to tackle this problem",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)


output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)
role = competency_data["role"]
company = competency_data["company"]

{
  "role": "Infrastructure Software Engineer",
  "company": "Y Combinator",
  "job_level": "6",
  "thinking": "The ideal candidate for Y Combinator's Infrastructure Software Engineer position should possess a strong technical background with experience in web application development and deployment, especially focused on the AWS ecosystem. Given Y Combinator's mission to support startups, the ability to work efficiently and adapt to rapidly changing environments is crucial. Additionally, aligning with Y Combinator's values, the candidate should display integrity, respect, accountability, and an action-oriented approach to problem-solving. Emphasis should be placed on both individual productivity and contributions to the wider team, reflecting the high-impact, startup-centric focus of the organization.",
  "competencies": [
    {
      "name": "Full Stack Web Application Proficiency",
      "description": "Strong expertise in designing, debugging, and managing full stack web application

## Review and improve the list of competencies

In [19]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and review the key competencies 
for a hiring team to look for in a candidate. The competencies should recover the knowledge, skills, abilities and other characteristics (KSAOs)
required for the job. Evaluate the provided list of competencies and improve them if necessary.
You will examine the provided competency framework with an eye towards comprehensiveness, relevance, and alignment with industry best practices. 
Your evaluation will focus on ensuring that the framework effectively covers all necessary areas for the {role} at {company} and 
incorporates the company's values wherever practical.
Junior or low-level jobs should have fewer competencies (e.g., 6) and more senior jobs should have more (e.g., 10).
Do not include acronyms or terminology that would not be understood by an industry professional not from the company.
Refer to the actual company name wherever possible rather than use a generic word such as 'organization' or 'company'.
Do NOT ever use the word 'technological' instead use 'technology' or 'technical'.

If you think the list of competencies is good, you can leave it as is.
If you think the list of competencies is bad, you can delete it and write a new one.
If you think the list of competencies is good but needs improvement, you can edit it.
If you think the list of competencies is good but needs more competencies, you can add more.
If you think the list of competencies is good but needs less competencies, you can delete some.
If you think the list of competencies is good but needs to be re-ordered, you can re-order them.
If you think the list of competencies is good but needs to be re-worded, you can re-word them.
If a description is aligned with a company value, you should say so.

The job description is as follows:
{job_description}

The current list of competencies is as follows:
{output}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the improved key competency list as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
      "improvements": "Improvements made (if any)"
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
      "improvements": "Improvements made (if any)"
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
      "improvements": "Improvements made (if any)"
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)


{
  "role": "Infrastructure Software Engineer",
  "company": "Y Combinator",
  "competencies": [
    {
      "name": "Technical Expertise in Full Stack Web Development",
      "description": "Solid expertise in the design, troubleshooting, and management of full-stack web applications, favoring those with experience in Ruby on Rails as part of a diverse technology stack.",
      "improvements": "Specified Ruby on Rails given its use in Y Combinator's tech stack."
    },
    {
      "name": "Advanced Cloud Infrastructure Capabilities",
      "description": "Proven depth of understanding and hands-on experience with AWS services, with a focus on building and maintaining robust cloud infrastructure solutions vital to startup operations.",
      "improvements": "Clarified the emphasis on practical experience with core AWS services."
    },
    {
      "name": "Strategic Security and Compliance Acumen",
      "description": "Demonstrated knowledge and application of security practices to pr

## Create Markdown for Competency Framework Introduction

In [20]:
markdown_competencies_intro = "\n"
counter = 1
for item in competency_data["competencies"]:
    markdown_competencies_intro += f"""
{counter}. **{item["name"]}**: {item["description"]}

"""
    counter += 1

markdown_competencies_intro = f"""

## Competency Framework

The following competencies have been identified for the {role} role at {company}:

{markdown_competencies_intro}

"""

display(Markdown(markdown_competencies_intro))



## Competency Framework

The following competencies have been identified for the Infrastructure Software Engineer role at Y Combinator:



1. **Technical Expertise in Full Stack Web Development**: Solid expertise in the design, troubleshooting, and management of full-stack web applications, favoring those with experience in Ruby on Rails as part of a diverse technology stack.


2. **Advanced Cloud Infrastructure Capabilities**: Proven depth of understanding and hands-on experience with AWS services, with a focus on building and maintaining robust cloud infrastructure solutions vital to startup operations.


3. **Strategic Security and Compliance Acumen**: Demonstrated knowledge and application of security practices to protect applications and data, with a commitment to safeguarding user trust in line with Y Combinator's integrity value.


4. **Developer Experience Optimization**: Experience improving developer tools and processes to streamline deployment cycles and foster innovation, resonating with Y Combinator's respect for enhancing productivity.


5. **Proficiency with Infrastructure as Code (IaC)**: Skillfulness in defining, automating, and managing infrastructure utilizing tools like Terraform, promoting scalable tech solutions congruent with Y Combinator's mission.


6. **Containerization and Orchestration Expertise**: Practical skill set with containerization using Docker and related orchestration platforms, advocating for agile development practices in fast-paced startup ecosystems.


7. **Performance Optimization Prowess**: Capability to identify and resolve performance issues, driving the efficiency and reliability essential to Y Combinator's productivity-focused ethos.


8. **Collaborative Spirit and Effective Communication**: Strong interpersonal and collaboration skills paired with clear and effective communication, enabling teamwork within a dynamic and high-impact engineering culture.


9. **Adaptable and Lean Approach to Problem Solving**: An entrepreneurial mindset that embraces quick learning cycles and adaptable problem-solving, in sync with Y Combinator's fast-paced, startup-centric environment.





## Evaluation scorecard markdown

In [21]:
markdown_evaluation_scorecard = f"""

## Candidate Evaluation

Complete the following scorecard for the {role} role at {company}. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
"""

spacer = " "

counter = 1
for item in competency_data["competencies"]:
    markdown_evaluation_scorecard += f"""| {counter}. {item["name"]} | {spacer} | {spacer} |
"""
    counter += 1

markdown_evaluation_scorecard += f"""| &nbsp; | | |
| **Total** | {spacer} | {spacer} |

### Other Interviewer Comments:

"""

display(Markdown(markdown_evaluation_scorecard))



## Candidate Evaluation

Complete the following scorecard for the Infrastructure Software Engineer role at Y Combinator. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
| 1. Technical Expertise in Full Stack Web Development |   |   |
| 2. Advanced Cloud Infrastructure Capabilities |   |   |
| 3. Strategic Security and Compliance Acumen |   |   |
| 4. Developer Experience Optimization |   |   |
| 5. Proficiency with Infrastructure as Code (IaC) |   |   |
| 6. Containerization and Orchestration Expertise |   |   |
| 7. Performance Optimization Prowess |   |   |
| 8. Collaborative Spirit and Effective Communication |   |   |
| 9. Adaptable and Lean Approach to Problem Solving |   |   |
| &nbsp; | | |
| **Total** |   |   |

### Other Interviewer Comments:



# Detailed Competencies

## Function for analyzing a competency in depth

In [22]:
def analyze_competency(competency, competency_description):
    prompt = f"""
You are an Interviewing and Assessment Expert. Your task is to do an analysis of the competency "{competency}" for a job to 
give to a hiring team. 
Do not use the word 'technological' instead use 'technology' or 'technical'.

You should generate a great job-relevant interview question to assess the knowledge, skills, abilities and other characteristics relevant to "{competency}". 
It should be the BEST question you could ask. 
The question should include 1-4 short follow-up questions embedded in the same paragraph. 
Also produce what to look for in the answer to the lead question when evaluating the candidate's answer.

You will then generate a scenario for "{competency}" to ask the candidate to respond to. The scenario should 50 to 75 words.

You will then generate 3 subcompetencies for the "{competency}" competency. Each subcompetency should have a name and a job-relevant interview
question to ask the candidate. Focus on aspects the knowledge, skills, abilities and other characteristics required for this competency
that are not already covered by the lead question.

You will then generate a set of VERY concise evaluation rubrics of less than 18 words to use when evaluating the candidate's answer 
to the lead question and the scenario.
Assume we are using a 5 point scale where 1 is the lowest rating and 5 is the highest rating.
Assume we know we are rating the candidate and don't refer to the candidate.

The competency description is currently as follows. Expand this description in your response if you think it is necessary to vividly describe the ideal candidate
but keep it to about 50 words:
{competency_description}

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the competency analysis list as only a JSON object with no other commentary. Use the following format:

{{"competency": [{{
  "name": "{competency}",
  "description": "{competency_description}",
  "lead_question": "Lead Interview Question",
  "lead_look_for": "What to look for and evaluate in the answer to the lead question. Address the interviewer.",
  "scenario": "A scenario to ask the candidate to respond to",
  "scenario_look_for": "What to look for or evaluate in the answer to the scenario. Address the interviewer.",
  "subcompetencies": [
    {{
      "name": "Subcompetency 1",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 2",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 3",
      "question": "Interview question for Subcompetency"
    }}
  ],
  "rating_1": "What a level 1 rating for this competency looks like. Be very concise.",
  "rating_2: "What a level 2 rating for this competency looks like. Be very concise.",
  "rating_3": "What a level 3 rating for this competency looks like. Be very concise.",
  "rating_4": "What a level 4 rating for this competency looks like. Be very concise.",
  "rating_5": "What a level 5 rating for this competency looks like. Be very concise."
}}"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return json.loads(output)


## Generate detailed competencies

This can take 3-5 minutes to generate.

In [23]:
# intialize the competency details object
competency_details = {"competencies":[]}

# set this to max number of competencies to generate. Use a lower number (e.g., 2) during testing as each competency takes 30+ seconds to generate
max_competencies = 11

# loop through the competencies and generate the competency details
for i, competency in enumerate(competency_data["competencies"]):
    if i == max_competencies:
        break
    competency_deep_dive = analyze_competency(competency["name"], competency["description"])
    competency_details["competencies"].append(competency_deep_dive)

{
  "competency": [{
    "name": "Technical Expertise in Full Stack Web Development",
    "description": "Proficiency in building and managing the full stack of web applications, particularly those involving Ruby on Rails within a diverse technology stack. Adept in a range of web technologies, including both front-end and back-end development, with the ability to troubleshoot complex issues across the stack and enhance application performance.",
    "lead_question": "Please describe a complex web application you have developed with Ruby on Rails. How did you ensure it was scalable and secure, and what was your approach to handling data persistence, API integration, and user interface challenges?",
    "lead_look_for": "Assess comprehensive understanding of Ruby on Rails stack, scalability and security strategies, familiarity with API integration, data management, and problem-solving related to UI.",
    "scenario": "Your Rails application is experiencing increased latency, causing user

## Create Interview Agenda

In [24]:
def generate_interview_agenda(role, company, job_description, company_description, competency_data):    
    prompt = f"""
You are an Interviewing and Assessment Expert. 
Your task is to do an analysis of the job description, the company description, and the competency framework to design an 
agenda for an interview team to use when interviewing a candidate for the {role} role at {company}.
Do not include interview questions, they will be generated later.
Do not include a description of the competencies, just include their name. Details about the competencies will be generated later.
Return the content as markdown text with absolutely no delimiters or commentary.

The agenda should have the following outline. Generate the appropriate text under each heading:
## Interview Agenda
### Welcome and Introduction
### Company Overview
### Job Overview
### Competency Deep Dive
### Candidate Vision and Strategy
### Q&A and Wrap Up

The job description is as follows:
{job_description}

The company description is as follows:
{company_description}

The competency framework is as follows:
{competency_data}
"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        # response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return output

interview_agenda = generate_interview_agenda(role, company, job_description, company_description, competency_data)

## Interview Agenda
### Welcome and Introduction
- Greet the candidate warmly and express gratitude for their interest in the role
- Introduce the interview team, including their names and roles at Y Combinator

### Company Overview
- Share Y Combinator’s mission to support startup creation and growth
- Discuss the vision for a world where startups are the norm and innovation thrives
- Highlight Y Combinator’s core values: integrity, respect, accountability, and commitment to founder success

### Job Overview
- Outline the role of Infrastructure Software Engineer and its importance to Y Combinator
- Discuss the technical environment, including Ruby on Rails, AWS, and React
- Talk about key responsibilities such as providing a stable platform, securing applications and data, improving developer tools, and shipping shared code

### Competency Deep Dive
- Technical Expertise in Full Stack Web Development
- Advanced Cloud Infrastructure Capabilities
- Strategic Security and Compliance Acum

## Competency detail sections markdown

In [25]:
counter = 1
markdown_competencies = "\n"

for competency_group in competency_details["competencies"]:
  for competency_item in competency_group["competency"]:

    markdown_competencies += "## Competency " + str(counter) + ". **" 
    markdown_competencies += competency_item["name"] + "**\n\n"

    markdown_competencies += competency_item["description"] + "\n\n"

    markdown_competencies += "### Lead Question\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["lead_question"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["lead_look_for"] + "_\n\n"

    markdown_competencies += "\n### Situational Scenario\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["scenario"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["scenario_look_for"] + "_\n\n"

    markdown_competencies += "### Diving Deeper\n\n"
    for subcompetency in competency_item["subcompetencies"]:
      markdown_competencies += "**" + subcompetency["name"] + "**: \"" + subcompetency["question"] + "\"\n\n"
      markdown_competencies += "&nbsp;\n\n"

    markdown_competencies += "\n### Evaluation\n\n"
    markdown_competencies += "| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |\n"
    markdown_competencies += "| -------- | -------- | -------- | -------- | -------- |\n"
    markdown_competencies += "|" + competency_item["rating_1"] + "|" + competency_item["rating_2"] + "|" + competency_item["rating_3"] + "|" + competency_item["rating_4"] + "|" + competency_item["rating_5"] + "|\n\n"
    
    # markdown_competencies += "\n### Notes\n\n&nbsp;\n\n"

    counter += 1




# Final markdown

In [26]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

January 12, 2024 2:51 PM


In [27]:
# Build final markdown

final_markdown = f"""--- 
title: Interview Guide for {role}
subtitle: {company}
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

# Interview Guide for {role}

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of {role}. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
{company}'s values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and {company}.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

{company_description}

{job_description}

{interview_agenda}

{markdown_competencies_intro}

{markdown_competencies}

{markdown_evaluation_scorecard}
"""

display(Markdown(final_markdown))

# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(final_markdown)

--- 
title: Interview Guide for Infrastructure Software Engineer
subtitle: Y Combinator
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 12, 2024 2:51 PM
--- 

# Interview Guide for Infrastructure Software Engineer

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of Infrastructure Software Engineer. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
Y Combinator's values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and Y Combinator.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

## Y Combinator Mission, Vision, and Values

### Mission

Y Combinator's mission is to increase the number of startups by assisting founders in their creation. The organization aims to get startups through their initial phase, often helping them build something impressive enough to raise significant amounts of money on a larger scale.

### Vision

Y Combinator envisions a world where startups are the norm rather than the exception. They believe that startups are, on balance, a good thing, as their founders and early employees can be much more productive than they would be working for an established company. Y Combinator's vision is to apply mass production techniques to startup funding, making the process more efficient and beneficial for founders.

### Values

Y Combinator values integrity, respect, and accountability. They believe these traits are critical for founders to possess. The organization is built on a network of trust, with its strength and value hinging on the trustworthiness of its members. Unethical behavior is not tolerated, as it puts the reputation of the entire community at risk. Y Combinator also values the idea of putting the founders' interests first, even before their own. This approach, while seemingly counterintuitive in a for-profit business, has proven to be more scalable and beneficial in the long run.

Y Combinator also values evidence of demand for a startup's solution and the strength of the founding team. They believe in the importance of launching and testing ideas quickly, rather than overplanning or overthinking. This approach is based on the understanding that the market and user needs can change rapidly, and startups need to be agile and responsive to these changes.

In summary, Y Combinator's mission, vision, and values revolve around fostering a supportive and ethical environment for startups, promoting efficiency and productivity, and encouraging quick action and responsiveness to market demands.


## Job Description

**Y Combinator - Infrastructure Software Engineer**

Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benefits and unlimited vacation – while working on a small, fast-moving and high-impact team.

In your first month, you might:

- Migrate a Rails application from Capistrano and EC2 to Docker and ECS
- Upgrade one of our applications to a newer version of Postgres or Rails
- Move more of our AWS infrastructure into Terraform
- Simplify our use of Webpack and Javascript for both developer productivity and faster deployments
- Create shared infrastructure to help manage email subscriptions and unsubscribes across 5 different product teams
- Triage and respond to bug bounty reports
- Work with outside security auditors to run a penetration test against our applications

**Key Responsibilities:**

- Provide a stable modern platform for our applications to be deployed to
- Keep our applications and data secure
- Improve our developer experience and tools so the YC Software Team can ship product updates quickly
- Help debug performance issues in our web applications
- Ship Ruby on Rails and TypeScript code that is shared among multiple YC applications

**Skills:**

- You know how to develop and debug MVC web applications. We use Ruby on Rails, Postgres, and React, but don’t worry if you haven’t used those specific technologies before
- You know how to deploy and run applications with AWS and Docker
- You use Terraform to make infrastructure changes

**Experience:**

- You have at least 5 years of experience developing and deploying web applications
- You are deeply familiar with the AWS platform specifically ECS, RDS, IAM, and CloudWatch

**Location:** YC is headquartered in the SF Bay Area with employees working from home or from our offices in San Francisco and Mountain View, CA. Candidates must live in the SF Bay Area or be willing to relocate.

**Compensation:** $130,000 to $270,000 + bonus (depending on skills and experience).

**Benefits:** Our full benefits package includes medical, vision, and dental plans, infertility benefit, STD/LTD, life insurance, commuter benefits, flexible spending account, health savings account, 401(k) + 4% matching, generous parental leave, paid holidays and flexible paid time off policy.

**Work Authorization:** This position does not support work authorization/visa sponsorship.

Y Combinator considers qualified applicants with criminal histories, consistent with applicable federal, state, and local law including San Francisco’s Fair Chance Ordinance. Y Combinator is committed to protecting the privacy of the personal information of job applicants and complying with the California Consumer Privacy Act. The privacy policy of Ashby, Inc., the hiring platform used by Y Combinator, governs the collection of such data.

## Interview Agenda
### Welcome and Introduction
- Greet the candidate warmly and express gratitude for their interest in the role
- Introduce the interview team, including their names and roles at Y Combinator

### Company Overview
- Share Y Combinator’s mission to support startup creation and growth
- Discuss the vision for a world where startups are the norm and innovation thrives
- Highlight Y Combinator’s core values: integrity, respect, accountability, and commitment to founder success

### Job Overview
- Outline the role of Infrastructure Software Engineer and its importance to Y Combinator
- Discuss the technical environment, including Ruby on Rails, AWS, and React
- Talk about key responsibilities such as providing a stable platform, securing applications and data, improving developer tools, and shipping shared code

### Competency Deep Dive
- Technical Expertise in Full Stack Web Development
- Advanced Cloud Infrastructure Capabilities
- Strategic Security and Compliance Acumen
- Developer Experience Optimization
- Proficiency with Infrastructure as Code (IaC)
- Containerization and Orchestration Expertise
- Performance Optimization Prowess
- Collaborative Spirit and Effective Communication
- Adaptable and Lean Approach to Problem Solving

### Candidate Vision and Strategy
- Invite the candidate to describe their vision for the role
- Explore how the candidate’s approach aligns with Y Combinator’s goals and values

### Q&A and Wrap Up
- Offer the candidate the opportunity to ask any questions about the role, team, or company
- Discuss the next steps in the hiring process
- Thank the candidate for their time and participation in the interview



## Competency Framework

The following competencies have been identified for the Infrastructure Software Engineer role at Y Combinator:



1. **Technical Expertise in Full Stack Web Development**: Solid expertise in the design, troubleshooting, and management of full-stack web applications, favoring those with experience in Ruby on Rails as part of a diverse technology stack.


2. **Advanced Cloud Infrastructure Capabilities**: Proven depth of understanding and hands-on experience with AWS services, with a focus on building and maintaining robust cloud infrastructure solutions vital to startup operations.


3. **Strategic Security and Compliance Acumen**: Demonstrated knowledge and application of security practices to protect applications and data, with a commitment to safeguarding user trust in line with Y Combinator's integrity value.


4. **Developer Experience Optimization**: Experience improving developer tools and processes to streamline deployment cycles and foster innovation, resonating with Y Combinator's respect for enhancing productivity.


5. **Proficiency with Infrastructure as Code (IaC)**: Skillfulness in defining, automating, and managing infrastructure utilizing tools like Terraform, promoting scalable tech solutions congruent with Y Combinator's mission.


6. **Containerization and Orchestration Expertise**: Practical skill set with containerization using Docker and related orchestration platforms, advocating for agile development practices in fast-paced startup ecosystems.


7. **Performance Optimization Prowess**: Capability to identify and resolve performance issues, driving the efficiency and reliability essential to Y Combinator's productivity-focused ethos.


8. **Collaborative Spirit and Effective Communication**: Strong interpersonal and collaboration skills paired with clear and effective communication, enabling teamwork within a dynamic and high-impact engineering culture.


9. **Adaptable and Lean Approach to Problem Solving**: An entrepreneurial mindset that embraces quick learning cycles and adaptable problem-solving, in sync with Y Combinator's fast-paced, startup-centric environment.






## Competency 1. **Technical Expertise in Full Stack Web Development**

Proficiency in building and managing the full stack of web applications, particularly those involving Ruby on Rails within a diverse technology stack. Adept in a range of web technologies, including both front-end and back-end development, with the ability to troubleshoot complex issues across the stack and enhance application performance.

### Lead Question

**&ldquo;Please describe a complex web application you have developed with Ruby on Rails. How did you ensure it was scalable and secure, and what was your approach to handling data persistence, API integration, and user interface challenges?&rdquo;**

&nbsp;

&nbsp;

_Assess comprehensive understanding of Ruby on Rails stack, scalability and security strategies, familiarity with API integration, data management, and problem-solving related to UI._


### Situational Scenario

**&ldquo;Your Rails application is experiencing increased latency, causing user complaints. Upon investigation, you discover an N+1 query issue. How would you go about solving this problem?&rdquo;**

&nbsp;

&nbsp;

_Candidate should exhibit ability to identify and resolve N+1 query problems, improving application efficiency._

### Diving Deeper

**Cloud Infrastructure Management**: "How would you go about migrating an existing Ruby on Rails application to AWS using Docker and ECS, while minimizing downtime?"

&nbsp;

**Security and Performance Optimization**: "Describe your approach to identifying and fixing a common security vulnerability in a web application and how you would optimize its performance."

&nbsp;

**Development Workflow Improvement**: "What tools and practices would you implement to enhance the productivity of a development team working on a Ruby on Rails project?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Limited knowledge, cannot describe a scalable or secure system.|Basic understanding, missing key concepts in scalability or security.|Adequate knowledge, addresses some but not all components of the stack.|Strong overview, clearly understands stack and scalability, minor gaps.|Expert analysis and troubleshooting of the full stack with advanced solutions.|

## Competency 2. **Advanced Cloud Infrastructure Capabilities**

Proven depth of understanding and hands-on experience with AWS services, essential for designing, implementing, and maintaining robust cloud infrastructure solutions critical to start-up operations.

### Lead Question

**&ldquo;Can you describe a complex cloud infrastructure you've designed or worked on, detailing the AWS services involved, the challenges you faced, and how you ensured its reliability and scalability? How would you monitor, troubleshoot, and optimize this infrastructure for performance and cost?&rdquo;**

&nbsp;

&nbsp;

_Look for detailed knowledge of AWS services, clarity in explaining the architecture, problem-solving and optimization skills, and continuous monitoring strategies._


### Situational Scenario

**&ldquo;Imagine you're tasked with migrating an entire suite of applications to AWS, prioritizing minimal downtime and data integrity. All the while, you have to ensure compliance and enhance performance. How would you proceed with this task?&rdquo;**

&nbsp;

&nbsp;

_Evaluate understanding of AWS migration strategies, awareness of compliance protocols, and planning for performance optimization and data integrity during migration._

### Diving Deeper

**System Reliability & Scalability**: "What strategies would you employ to maintain high availability and fault tolerance of an AWS-based infrastructure?"

&nbsp;

**Infrastructure as Code & Automation**: "How do you leverage infrastructure as code, such as Terraform, to manage and version cloud resources, and could you explain an instance where automation saved significant time or resources?"

&nbsp;

**Security & Compliance**: "Discuss how you ensure security and compliance within AWS cloud infrastructure and share an example of how you successfully navigated these requirements in a previous project."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Limited or no experience with AWS. Can’t describe a cloud infrastructure.|Basic AWS knowledge. Struggles with complex designs or optimization.|Competent with AWS services. Describes an effective cloud solution.|Strong in AWS architecture design and problem-solving. Offers optimization ideas.|Expert-level proficiency. Articulates an advanced, optimized AWS architecture.|

## Competency 3. **Strategic Security and Compliance Acumen**

Demonstrated knowledge of best practices in security and compliance, with a proactive approach in applying these to safeguard applications, data, and maintaining user trust, aligned with Y Combinator's commitment to integrity and high ethical standards.

### Lead Question

**&ldquo;Can you describe a time when you strategically implemented a security framework or compliance protocol within an infrastructure environment? What were the key considerations taken for the choice of framework and how did you ensure it aligned with the organizational goals and user trust?&rdquo;**

&nbsp;

&nbsp;

_Evaluate candidate's strategic thought process, familiarity with security frameworks, alignment of security protocols with organizational goals, and user trust maintenance._


### Situational Scenario

**&ldquo;You discover a critical security flaw in the AWS infrastructure configuration that could potentially expose sensitive user data. How would you address it, considering both immediate actions and long-term preventive strategies?&rdquo;**

&nbsp;

&nbsp;

_Assess problem-solving skills, immediate and strategic response to threats, and ability to implement long-term security improvements._

### Diving Deeper

**Security Framework Application**: "What factors do you consider when selecting and applying a security framework to an existing infrastructure?"

&nbsp;

**Regulatory Compliance Understanding**: "How do you stay up-to-date with regulatory compliance changes, and could you give an example of how you have adapted to such a change?"

&nbsp;

**Risk Management and Mitigation**: "Explain your process for identifying, assessing, and mitigating risks in software infrastructure."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows little understanding of security; vague, incorrect responses.|Basic understanding with minimal practical application; simplistic responses.|Solid grasp, moderate experience; response shows practical application.|Strong in-depth knowledge; well-rounded, strategic examples provided.|Expert-level; demonstrates strategic foresight, innovation in security.|

## Competency 4. **Developer Experience Optimization**

Skilled in enhancing developer tools and workflow efficiency, optimizing the development environment to accelerate deployment while encouraging innovation and tech adaptability.

### Lead Question

**&ldquo;Tell me about a time when you improved a development tool or process that resulted in faster deployment cycles and increased innovation. How did you measure the impact, and what feedback did you receive from the development team?&rdquo;**

&nbsp;

&nbsp;

_Look for specific examples, understanding of impact measurement, ability to elicit and incorporate feedback, and direct outcomes of initiatives._


### Situational Scenario

**&ldquo;Imagine our deployment cycle is slow due to outdated processes, and developers are getting frustrated. How would you identify the bottlenecks and which steps would you take to address them?&rdquo;**

&nbsp;

&nbsp;

_Expect problem identification skills, a systematic approach to optimization, and clear, actionable steps._

### Diving Deeper

**Workflow Efficiency**: "Can you provide an example of how you've configured a continuous integration/continuous deployment (CI/CD) pipeline to maximize developer efficiency?"

&nbsp;

**Tools Adaptability**: "How do you stay current with emerging development tools, and what's your approach to introducing and integrating these within an existing team?"

&nbsp;

**Innovation Facilitation**: "Describe an environment or process you have created that spurred innovation among developers. What were the key elements?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|No relevant experience or understanding of developer experience optimization.|Limited understanding; few examples of impact on deployment cycles.|Adequate impact examples; moderate improvements in deployment cycles.|Strong improvement examples; measures impact well; good team feedback.|Exceptional optimization initiatives with metrics; significant deployment acceleration; excellent team feedback.|

## Competency 5. **Proficiency with Infrastructure as Code (IaC)**

Expertise in creating, optimizing, and managing resilient cloud infrastructure utilizing IaC tools such as Terraform, aligned with promoting scalable technology solutions and Y Combinator's goal of assisting startups.

### Lead Question

**&ldquo;Describe a complex infrastructure you've managed with Infrastructure as Code tools. How did you ensure it was both scalable and maintainable? Can you discuss a specific challenge you encountered with IaC and how you overcame it?&rdquo;**

&nbsp;

&nbsp;

_Look for details on infrastructure complexity, scalability, maintainability, problem resolution, and use of IaC best practices._


### Situational Scenario

**&ldquo;Imagine our AWS infrastructure is experiencing sporadic outages, and your task is to use Terraform to implement a more robust architecture. How do you approach this?&rdquo;**

&nbsp;

&nbsp;

_Assess understanding of architecting for high availability, investigative approach, and practical experience with Terraform._

### Diving Deeper

**Technical Knowledge of IaC Tools**: "What are the advantages of using Terraform over other IaC tools, and what specific features do you utilize most frequently?"

&nbsp;

**IaC Security Practices**: "How do you ensure security best practices within your IaC configurations?"

&nbsp;

**Collaboration and Code Management**: "Can you explain how you manage state files and collaborate with team members when using IaC tools like Terraform?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Little to no relevant experience; major misunderstandings.|Basic familiarity; significant guidance needed.|Moderate experience; some aspects well-understood.|Strong, practical experience; minor gaps in knowledge.|Expert-level; demonstrates full mastery and innovation.|

## Competency 6. **Containerization and Orchestration Expertise**

Essential expertise in containerization using Docker, orchestrating with platforms like Kubernetes, supporting scalable, secure and resilient infrastructure in a startup environment, with agility to respond to rapidly evolving tech needs.

### Lead Question

**&ldquo;Can you describe a complex system you've worked on that leverages both containerization and orchestration? In your explanation, highlight how you managed service discovery, scaling, and zero-downtime deployment.&rdquo;**

&nbsp;

&nbsp;

_Look for detailed understanding of system architecture, practical problem-solving approaches, experience with service discovery mechanisms, scaling strategies, and deployment techniques ensuring minimal disruption._


### Situational Scenario

**&ldquo;Imagine you are tasked with deploying a microservices-based application on Kubernetes, where each service has varying resource requirements. Describe your approach to resource allocation and scaling.&rdquo;**

&nbsp;

&nbsp;

_Expect strategic thinking in managing resources, awareness of Kubernetes features for horizontal/vertical scaling, and ability to articulate a Kubernetes-centric deployment approach._

### Diving Deeper

**Docker Proficiency**: "Explain how you optimize Dockerfile for efficient image building and how you manage image versions in a continuous integration pipeline."

&nbsp;

**Kubernetes Ecosystem Familiarity**: "Discuss how you monitor and troubleshoot a Kubernetes cluster. Which tools do you prefer and why?"

&nbsp;

**Infrastructure as Code Expertise**: "Describe your experience with infrastructure as code tools such as Terraform. How do you manage state and changes in a collaborative environment?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Limited understanding; no hands-on experience.|Basic understanding; sparse hands-on experience.|Good knowledge; solid practical experience.|Advanced skill set; extensive experience.|Expert-level; leader in container and orchestration.|

## Competency 7. **Performance Optimization Prowess**

Capability to identify and resolve performance bottlenecks, enhancing system efficiency and dependability, critical to Y Combinator's productivity-centric philosophy.

### Lead Question

**&ldquo;Describe a time when you had to identify and resolve a complex performance bottleneck in a web application. What diagnostic tools did you use and how did your solution improve system performance? Can you also share the process you followed to ensure that the optimization did not adversely affect other system functionalities?&rdquo;**

&nbsp;

&nbsp;

_Seek evidence of effective use of diagnostic tools, a methodical approach to problem-solving, the ability to balance performance with system stability, and communication of the impact on overall functionality._


### Situational Scenario

**&ldquo;Your monitoring tools have alerted you to a sudden spike in server load and response times during peak usage of a web application. Outline the steps you would take to diagnose and address this issue.&rdquo;**

&nbsp;

&nbsp;

_Expect a structured approach to incident response, aptitude in identifying performance issues, and strategies to resolve the issue without significant downtime._

### Diving Deeper

**Diagnostic Tool Proficiency**: "What is your go-to set of tools for diagnosing and monitoring web application performance and why do you prefer them over others?"

&nbsp;

**Performance-Driven Development**: "Can you discuss how you anticipate performance needs and proactively integrate optimization within the development lifecycle?"

&nbsp;

**System Reliability Assurance**: "How do you ensure that performance optimizations do not compromise the reliability of a system? Please provide an example."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Inadequate understanding of diagnostic tools and processes.|Basic use of tools; limited problem-solving shown.|Adequate problem-solving; average use of standard tools.|Proficient in tools usage; good problem-solving and impact communication.|Expert in performance optimization with innovative and effective solutions.|

## Competency 8. **Collaborative Spirit and Effective Communication**

Strong interpersonal and collaboration skills paired with clear and effective communication, enabling teamwork within a dynamic and high-impact engineering culture. This includes the ability to concisely convey complex technical concepts, actively listen to team members, and adapt communication style to different audiences to facilitate mutual understanding and cooperation.

### Lead Question

**&ldquo;Can you tell us about a time when you had to work on a team to solve a complex problem? How did you ensure that your communication was clear and effective, and how did you manage different viewpoints to arrive at a solution?&rdquo;**

&nbsp;

&nbsp;

_Look for specific instances where the candidate demonstrated active listening, clarity in technical communications, ability to mediate conflicts, and a focus on collaborative outcomes._


### Situational Scenario

**&ldquo;Imagine you're leading a project to migrate an application to a new AWS service. Midway, an issue causes differing opinions between teams. How would you align the team towards a common goal?&rdquo;**

&nbsp;

&nbsp;

_Evaluate the candidate's approach to conflict resolution, ability to communicate technical details clearly, and strategies for fostering a cooperative team environment._

### Diving Deeper

**Active Listening and Feedback**: "Describe a time when you received unexpected feedback during a project. How did you process and respond to it?"

&nbsp;

**Conflict Resolution and Mediation Skills**: "How would you handle a situation where two colleagues have conflicting technical solutions to a problem?"

&nbsp;

**Adaptability in Communication**: "Share an example of how you've adjusted your communication style when explaining technical issues to non-technical stakeholders."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Does not demonstrate understanding of collaboration or effective communication.|Has basic collaborative skills but struggles with clear communication.|Shows collaborative efforts and communicates effectively, yet room for refinement.|Strong collaboration and communication; occasional lapses in conflict resolution.|Exemplary collaborator, communicator; seamlessly resolves conflicts, aligns teams.|

## Competency 9. **Adaptable and Lean Approach to Problem Solving**

Seeks efficient and flexible problem-solving methods that maximize learning and minimize waste, aligning with the rapid iteration and innovation required in a startup environment.

### Lead Question

**&ldquo;Can you describe an instance where you identified a significant technical challenge and how you adapted your strategy to solve it efficiently? What was the outcome, and what did you learn from that situation?&rdquo;**

&nbsp;

&nbsp;

_Look for concrete examples demonstrating adaptability, efficient problem-solving, the ability to learn quickly from actions taken, and evidence of iteration leading to a successful outcome._


### Situational Scenario

**&ldquo;Your deployment pipeline has suddenly started failing due to an unexpected AWS service outage. How would you address this to minimize downtime for active product deployments?&rdquo;**

&nbsp;

&nbsp;

_The candidate's answer should show practical technical knowledge, prioritization of immediate actions, strategic long-term solutions, and ability to communicate with the team._

### Diving Deeper

**Rapid Learning Ability**: "How do you stay updated with the latest changes in AWS services, and could you provide an example when this knowledge significantly impacted a project?"

&nbsp;

**Efficiency in Debugging**: "What is your approach to debugging a complex system that you are not fully familiar with, and how do you ensure minimal impact on existing operations?"

&nbsp;

**Communication Under Pressure**: "Tell me about a time when you had to communicate a critical technical issue to a non-technical stakeholder. How did you ensure they understood the implications and urgency?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Demonstrates inability to adapt or iterate effectively.|Shows basic adaptation but with limited efficiency.|Adapts to changes, with moderate efficiency and learning.|Adapts well, learns quickly, and improves processes.|Excels in adaptability, rapid learning, and driving efficient outcomes.|





## Candidate Evaluation

Complete the following scorecard for the Infrastructure Software Engineer role at Y Combinator. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
| 1. Technical Expertise in Full Stack Web Development |   |   |
| 2. Advanced Cloud Infrastructure Capabilities |   |   |
| 3. Strategic Security and Compliance Acumen |   |   |
| 4. Developer Experience Optimization |   |   |
| 5. Proficiency with Infrastructure as Code (IaC) |   |   |
| 6. Containerization and Orchestration Expertise |   |   |
| 7. Performance Optimization Prowess |   |   |
| 8. Collaborative Spirit and Effective Communication |   |   |
| 9. Adaptable and Lean Approach to Problem Solving |   |   |
| &nbsp; | | |
| **Total** |   |   |

### Other Interviewer Comments:




## Build Word docx

In [28]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [29]:
# Convert from markdown to HTML

!pandoc -s {markdown_destination} -o {html_destination} --css=../../templates/html-styles.css --reference-doc="templates/html-reference.html" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [30]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['templates/html-styles.css'], full_fonts=True, hinting=True, presentational_hints=True)